In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import cv2 
import os
import pickle
import glob
import math
import random
!pip install imutils
import imutils
import pickle
import gc

from tqdm import tqdm
import keras_tuner as kt
from tensorflow.keras.utils import Sequence
import tensorflow.keras.backend as K
import pathlib
from sklearn.model_selection import StratifiedKFold

DATASET_PATH = r"../input/neoplasie/multimodalMRI/"
CATEGORIES = ["ADC","DWI","FLAIR","GRE","MDC","T1W"]

In [ ]:
def deskull(path, IMG_SIZE, direct_mode = False):
    if not direct_mode:
        img = cv2.imread(path)
    else:
        img = cv2.cvtColor(path, cv2.COLOR_BGR2RGB)

    img = cv2.resize(
                img,
                dsize=IMG_SIZE,
                interpolation=cv2.INTER_CUBIC
        )
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # threshold the image, then perform a series of erosions +
    # dilations to remove any small regions of noise
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)

    # find the extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])

    # add contour on the image
    img_cnt = cv2.drawContours(img.copy(), [c], -1, (0, 255, 255), 4)

    # add extreme points
    img_pnt = cv2.circle(img_cnt.copy(), extLeft, 8, (0, 0, 255), -1)
    img_pnt = cv2.circle(img_pnt, extRight, 8, (0, 255, 0), -1)
    img_pnt = cv2.circle(img_pnt, extTop, 8, (255, 0, 0), -1)
    img_pnt = cv2.circle(img_pnt, extBot, 8, (255, 255, 0), -1)

    # crop
    ADD_PIXELS = 0
    img2 = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
    img_final = cv2.resize(
        img2,
        dsize=IMG_SIZE,
        interpolation=cv2.INTER_CUBIC
    )
    return img_final.copy()

In [ ]:
def read_dataset(dataframe, indexes=[], perform_deskull=True):
    X_train = []
    Y_train = []
    for category in CATEGORIES:
        X_train.append([])

    if len(indexes)>0:
        source = indexes
    else:
        source = dataframe.index
    
    for index in source:
        row = dataframe.iloc[index]
        temp = []
        for category in CATEGORIES:
            cat_index = CATEGORIES.index(category)
            current_path = pathlib.PureWindowsPath(row[category]).as_posix()
            if perform_deskull:
                X_train[cat_index].append(deskull(DATASET_PATH + current_path,(224,224))/255)
            else:
                image = cv2.imread(DATASET_PATH + current_path)
                image = cv2.resize(image, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
                X_train[cat_index].append(image/255)
        if row["Class"] == 'MET':
            Y_train.append(1)
        else:
            Y_train.append(0)

    for i in range(0,len(CATEGORIES)):
        X_train[i] = np.array(X_train[i])
    return np.array(X_train), np.array(Y_train)

In [ ]:
dataframe = pd.read_csv(DATASET_PATH + 'dataset.csv')
folds = pickle.load(open(DATASET_PATH + 'folds.npy', "rb"))

In [ ]:
#SHUFFLE
for i in range(0,len(folds)):
    indices = np.arange(folds[i].shape[0])
    np.random.shuffle(indices)
    folds[i] = folds[i][indices]

In [ ]:
X_train, Y_train = read_dataset(dataframe, [*folds[8], *folds[9], *folds[2], *folds[3], *folds[4], *folds[5], *folds[6], *folds[7]], perform_deskull=True) #Fold 2 to 9 //80%
X_val, Y_val = read_dataset(dataframe, [*folds[0],*folds[1]], perform_deskull=True) #Fold 0,1 //20%

1. # Architettura Rete

In [ ]:
import tensorflow
import keras
from keras import Input, Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, Flatten, Concatenate
from keras.activations import relu
from keras import backend as K

#Modello
def get_model():
    # Multiple inputs
    in1 = Input(shape=(224,224,3))
    in2 = Input(shape=(224,224,3))
    in3 = Input(shape=(224,224,3))
    in4 = Input(shape=(224,224,3))
    in5 = Input(shape=(224,224,3))
    in6 = Input(shape=(224,224,3))

    # CNN output
    cnn = DenseNet121(include_top=False)
    cnn.trainable = False

    out1 = cnn(in1)
    out2 = cnn(in2)
    out3 = cnn(in3)
    out4 = cnn(in4)
    out5 = cnn(in5)
    out6 = cnn(in6)

    # Flattening the output for the dense layer
    fout1 = Flatten()(out1)
    fout2 = Flatten()(out2)
    fout3 = Flatten()(out3)
    fout4 = Flatten()(out4)
    fout5 = Flatten()(out5)
    fout6 = Flatten()(out6)

    # Concatenating the final output
    out = Concatenate(axis=-1)([fout1, fout2, fout3, fout4, fout5, fout6])
    out = Flatten()(out)

    # Getting the dense output
    dense = Dense(32, activation='relu')(out)
    dense = Dense(1, activation='sigmoid')(dense)

    # Creating the model
    model = Model(inputs=[in1,in2,in3,in4,in5,in6], outputs=dense)
    return model

#Metriche
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
keras.utils.plot_model(model)

# Addestramento

In [ ]:
#Optional Callback: save checkpoints
callbacks = []
runPath = './models/'
#callbacks.append(keras.callbacks.ModelCheckpoint(runPath + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only = False, mode='min', save_freq='epoch'))

In [ ]:
#keras.optimizers.RMSprop(lr=1e-4)
model = get_model()
model.compile(optimizer=keras.optimizers.Adam(lr=0.00001), loss=tf.keras.losses.BinaryCrossentropy(),metrics=['binary_accuracy',f1_m,precision_m, recall_m])
history = model.fit([X_train[0], X_train[1], X_train[2], X_train[3], X_train[4], X_train[5]], Y_train, epochs=10, validation_data= ([X_val[0], X_val[1], X_val[2], X_val[3], X_val[4], X_val[5]], Y_val))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sn

labels = ["GBM (0)","MET (1)"]

#Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
Y_test = Y_val
y_pred = model.predict([X_val[0], X_val[1], X_val[2], X_val[3], X_val[4], X_val[5]])
rounded_pred = np.round(abs(y_pred))
print(classification_report(Y_test, rounded_pred, target_names=labels))

confusion_matrix = confusion_matrix(Y_test, rounded_pred)

sn.set(font_scale = 1.4)
sn.heatmap(confusion_matrix, annot=True, annot_kws={"size":16})
plt.show()

In [ ]:
model = keras.models.load_model("../input/neoplasiemodels/MULTI/VGG16 MULTI 80-20/best_model.h5", custom_objects={"recall_m": recall_m, "precision_m": precision_m, "f1_m": f1_m})

In [ ]:
def predict(model, X, Y, index):
    prediction = model.predict([X[0][index:index+1], X[1][index:index+1], X[2][index:index+1], X[3][index:index+1], X[4][index:index+1], X[5][index:index+1]])
    gt = Y[index]
    return prediction, gt

In [ ]:
predict(model,X_val,Y_val,0)

In [ ]:
plt.figure()
plt.imshow(X_train[0][0])

In [ ]:
#Plot a video della training history
plt.figure(figsize=(20, 10))
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.figure(figsize=(20, 10))
# summarize history for loss
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# SALVATAGGIO MODELLO:
model.save('./best_model.h5')

#Salvo i dati "storici" completi dell'apprendimento
with open('./best_history.npy', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

In [ ]:
model.evaluate(X_val)